In [ ]:
DATABASE_ADDRESS = "../Data/Raw/chartevents.csv"
PROCEDURE_EVENTS_ADDRESS = "../Data/Raw/procedureevents.csv"
CODE_MAPPING_ADDRESS = "../Data/Raw/code_mappings.csv"
DATA_SIZE = 100*1000000

###### 20大概需要14分鐘
###### 1需要30.5MB
###### 大檔案有 329,499789行

In [ ]:
FILENAME = datetime.datetime.now().strftime('%Y%m%d') + f'_{DATA_SIZE:.0e}'
FILENAME

In [ ]:
item = dict()
mark = tuple()

In [ ]:
df = pd.read_csv(CODE_MAPPING_ADDRESS, engine="python", encoding="unicode_escape")
id_to_label = dict(zip(df["itemid"], df["label"]))
label_to_id = dict(zip(df["label"], df["itemid"]))
# df.head()

In [ ]:
mark = ("Heart Rate", \
        "Respiratory Rate", \
        "Non Invasive Blood Pressure systolic", \
        "Non Invasive Blood Pressure diastolic", \
        "O2 saturation pulseoxymetry", \
        "Temperature Celsius", \
        "Temperature Fahrenheit")

In [ ]:
%%time 
df = pd.read_csv(DATABASE_ADDRESS, usecols=["hadm_id", "charttime", "itemid", "value"], engine="python", encoding="unicode_escape", nrows=DATA_SIZE)
df.info()

In [ ]:
df['charttime'] = pd.to_datetime(df['charttime']).round("10min")

In [ ]:
# 建立一個CLASS容器來裝每一個PATIENT
class Patient:
    data = dict()
    def __init__(self, ):
        self.data = {"subject_id":None, "hadm_id":None, "charttime":None}
        for label_ in mark:
            self.data[label_] = None
    def allna(self, ):
        return all ([self.data[i]==None for i in mark])

In [ ]:
output = []
cnt = 0
tp = [label_to_id[i] for i in mark]
for name, group in df.groupby(["hadm_id", "charttime"]):
    patient = Patient()
    for id_, value_ in zip(group.itemid, group.value):
        cnt += 1
        if id_ in tp:
            patient.data[id_to_label[id_]] = value_
    if not patient.allna():
    # if True:
        patient.data["hadm_id"], patient.data["charttime"] = name
        if patient.data["Temperature Fahrenheit"] != None:
            patient.data["Temperature Celsius"] = round((float(patient.data["Temperature Fahrenheit"]) - 32) * 5 / 9, 1)
        output.append(patient.data)
merged_df = pd.DataFrame(output,columns=['hadm_id','charttime']+list(mark)).drop(["Temperature Fahrenheit"], axis=1)
merged_df.to_csv(f"../Data/Preprocessed/chartevents_{FILENAME}_pivot.csv", float_format='%.2f', index=0)

In [ ]:
# df.to_csv("chart_test3_merge_new.csv", float_format='%.2f', index=0)
print("amount: ", len(output))
print("amount: ", len(merged_df))
# merged_df

In [ ]:
# ------------------------TODO------------------------ #

In [ ]:
testdf = merged_df

In [ ]:
testdf.sample()

In [ ]:
new_mark = ("Heart Rate", \
        "Respiratory Rate", \
        "Non Invasive Blood Pressure systolic", \
        "Non Invasive Blood Pressure diastolic", \
        "O2 saturation pulseoxymetry", \
        "Temperature Celsius")
testdf[2:3][list(new_mark)]

In [ ]:
df = pd.read_csv(f"../Data/Preprocessed/chartevents_{FILENAME}_pivot.csv", engine="python", encoding="unicode_escape", parse_dates=["charttime"])
df.dropna(thresh=5, inplace=True)
for name, group in df.groupby("hadm_id"):
    group.interpolate(method='bfill', inplace=True)
    group.interpolate(method='ffill', inplace=True)
    df.update(group)
df.to_csv(f"../Data/Preprocessed/chartevents_{FILENAME}_filled.csv", float_format='%.1f', index=0)

In [ ]:
df.info()

In [ ]:
def read_csv_function(road: str) -> pd.core.frame.DataFrame:
    print("Reading", road, "...")
    return pd.read_csv(road, engine='python')
def to_timestamp(s: str):
    try:
        return int(mktime(strptime(s, "%Y-%m-%d %H:%M:%S")))
    except ValueError:
        try:
            return int(mktime(strptime(s, "%Y-%m-%d %H:%M")))
        except ValueError:
            return int(mktime(strptime(s, "%Y/%m/%d %H:%M")))

In [ ]:
df = read_csv_function(PROCEDURE_EVENTS_ADDRESS)
dic = {}
for i, it in enumerate(df.itemid):
    if it != 225466:
        continue
    if df.iloc[i].hadm_id not in dic:
        dic[df.iloc[i].hadm_id] = list()
    dic[df.iloc[i].hadm_id].append(to_timestamp(str(df.iloc[i].starttime)))

df = read_csv_function(f"../Data/Preprocessed/chartevents_{FILENAME}_filled.csv")
ans = [False]*len(df)
for i in range(len(df)):
    if int(df.iloc[i].hadm_id) in dic:
        for j in dic[df.iloc[i].hadm_id]:
            if 0 <= j - to_timestamp(str(df.iloc[i].charttime)) <= 14400:
                ans[i] = True
                break

df["label"] = ans
df.to_csv(f"../Data/Preprocessed/chartevents_{FILENAME}_labeled.csv", float_format='%.2f', index=0)

In [ ]:
df.label.value_counts()